# Tutorial 1.7: Evaluating Agents with MLflow

## LLM-as-Judge Evaluation for GenAI Applications

This notebook teaches you how to systematically evaluate agents and LLM applications using MLflow's evaluation framework. You'll learn to use built-in judges, create custom scorers, and integrate third-party evaluation libraries like DeepEval.

### What You'll Learn

- ✅ Why agent evaluation matters
- ✅ MLflow built-in scorers (RelevanceToQuery, Correctness, Guidelines, Safety)
- ✅ Creating custom scorers with the `@scorer` decorator
- ✅ DeepEval integration for conversational evaluation
- ✅ Session-level multi-turn evaluation

### Prerequisites
- Completed notebooks 1.1-1.6
- Understanding of MLflow tracing

### Estimated Time: 25-30 minutes

---
## Step 1: Why Evaluate Agents?

Agent evaluation is critical for:

| Challenge | Solution |
|-----------|----------|
| Non-deterministic outputs | Statistical evaluation over multiple runs |
| Complex reasoning chains | Step-by-step quality assessment |
| Multi-turn conversations | Session-level coherence metrics |
| Safety and compliance | Automated guardrail checking |
| Regression detection | Baseline comparisons |

### LLM-as-Judge Pattern

Instead of brittle string matching, we use LLMs to evaluate LLM outputs:

```
Agent Output → Judge LLM → Score + Reasoning
```

![LLM as a judge concept](images/llm-as-judge.png)

MLflow provides:
1. **Built-in scorers** - Pre-configured judges for common metrics
2. **Custom scorers** - Define your own evaluation logic
3. **Third-party integrations** - DeepEval, RAGAS, Phoenix, and more

---
## Step 2: Environment Setup

In [2]:
import mlflow
from dotenv import load_dotenv
from utils.clnt_utils import is_databricks_ai_gateway_client, get_databricks_ai_gateway_client, get_openai_client, get_ai_gateway_model_names

# Load environment
load_dotenv()

# Configure MLflow
EXPERIMENT_NAME = "11-agent-evaluation"
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(EXPERIMENT_NAME)

#Check if we are using a Databricks AI Gateway client
use_databricks_provider = is_databricks_ai_gateway_client()
if use_databricks_provider:
    client = get_databricks_ai_gateway_client()
    models = get_ai_gateway_model_names()
    JUDGE_MODEL = models[2]
    AGENT_MODEL = models[0]
else:
    # Initialize as an OpenAI client
    client = get_openai_client()
    JUDGE_MODEL = "gpt-5.2"
    AGENT_MODEL = "gpt-5.2"

# Enable autologging
mlflow.openai.autolog()

print("✅ Environment configured for agent evaluation")
print(f"use_databricks_provider: {use_databricks_provider}")
print(f"MLflow tracking: {mlflow.get_tracking_uri()}")
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Agent model : {AGENT_MODEL}")
print(f"Judge model : {JUDGE_MODEL}")

✅ Environment configured for agent evaluation
use_databricks_provider: False
MLflow tracking: http://localhost:5000
Experiment: 11-agent-evaluation
Agent model : gpt-5.2
Judge model : gpt-5.2


---
## Step 3: Create a Simple Agent for Evaluation

Let's create a simple Q&A agent that we'll evaluate. This is a good first stepping stone to the next steps in which we use a number of built-in judges, followed by custom scorers.

In [3]:
from typing import Any

class SimpleQAAgent:
    """
    A simple Q&A agent for demonstration.
    """
    
    def __init__(self, client:Any, model: str = "gpt-5.2"):
        self.model = model
        self.client = client
    
    @mlflow.trace(name="qa_agent", span_type="AGENT")
    def answer(self, question: str) -> str:
        """Answer a question."""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": """You are a helpful Agent Assistant. Provide concise, accurate answers, 
                no hallucinations, with a focus on MLflow and GenAI."""},
                {"role": "user", "content": question}
            ],
            temperature=0.7,
        )
        return response.choices[0].message.content

# Initialize agent
agent = SimpleQAAgent(client=client, model=AGENT_MODEL)

# Test the agent with a single question
test_question = "What is MLflow for GenAI?"
test_response = agent.answer(test_question)
print(f"Question: {test_question}")
print(f"\nResponse: {test_response}")

Question: What is MLflow for GenAI?

Response: MLflow for GenAI is MLflow’s set of features for **building, tracking, evaluating, and deploying GenAI applications**—especially those involving **LLMs**, **prompting**, **retrieval-augmented generation (RAG)**, and **agentic workflows**.

Key capabilities typically include:

- **Experiment tracking for GenAI runs**  
  Log prompts, model/provider info, parameters (e.g., temperature), tool calls, retrieved context, and outputs—so LLM app behavior is reproducible and comparable.

- **Prompt and artifact management**  
  Version and store prompts/templates, system instructions, example sets, and other LLM app assets alongside runs.

- **Evaluation tailored to LLMs**  
  Run offline evaluations using built-in or custom metrics (e.g., relevance, groundedness, toxicity, faithfulness), including LLM-as-a-judge patterns and dataset-based evals.

- **Model packaging and deployment**  
  Package GenAI apps (including chains/RAG pipelines) into depl

Trace(trace_id=tr-2cfdf759552f271911eae1f56da08197)

### Quick Evaluation: Is This Response Good?

Now that we have a response, how do we know if it's good? We could read it manually, but that doesn't scale. Instead, let's use MLflow's built-in `RelevanceToQuery` scorer to have an LLM judge evaluate the response.

In [4]:
import os
from mlflow.genai.scorers import RelevanceToQuery

# Configure judge model URI based on provider
if use_databricks_provider:
    databricks_token = os.environ.get("DATABRICKS_TOKEN")
    ai_gateway_base_url = os.environ.get("AI_GATEWAY_BASE_URL")
    os.environ["OPENAI_API_KEY"] = databricks_token
    os.environ["OPENAI_API_BASE"] = ai_gateway_base_url
    judge_model_uri = f"openai:/{JUDGE_MODEL}"
else:
    judge_model_uri = f"openai:/{JUDGE_MODEL}"

# Create a quick evaluation of the single test response
quick_eval_data = [{
    "inputs": {"question": test_question},
    "outputs": {"response": test_response}  # Pre-computed output, no predict_fn needed
}]

# Evaluate with RelevanceToQuery scorer
quick_scorer = RelevanceToQuery(model=judge_model_uri)

print("🔄 Evaluating the test response with RelevanceToQuery scorer...\n")

quick_result = mlflow.genai.evaluate(
    data=quick_eval_data,
    scorers=[quick_scorer]
)

# Show the result
print("📊 Quick Evaluation Result:")
print("-" * 40)
score = quick_result.metrics.get("relevance_to_query/mean", "N/A")
print(f"   Relevance Score: {score}")
print("\n✅ The LLM judge evaluated our agent's response!")
print("   Now let's learn about all the built-in scorers available...")

2026/02/06 11:56:04 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


🔄 Evaluating the test response with RelevanceToQuery scorer...



Evaluating:   0%|          | 0/1 [Elapsed: 00:00, Remaining: ?] 

📊 Quick Evaluation Result:
----------------------------------------
   Relevance Score: 1.0

✅ The LLM judge evaluated our agent's response!
   Now let's learn about all the built-in scorers available...


---
## Step 4: MLflow Built-in Scorers

MLflow provides [pre-configured scorers](https://mlflow.org/docs/latest/genai/eval-monitor/scorers/llm-judge/predefined/) for common evaluation needs:

| Scorer | Purpose | Inputs |
|--------|---------|--------|
| `RelevanceToQuery` | Is the response relevant to the question? | inputs, outputs |
| `Correctness` | Is the response factually correct? | outputs, expectations |
| `Guidelines` | Does the response follow specific guidelines? | outputs |
| `Safety` | Is the response safe and appropriate? | outputs |

In [9]:
import os
from mlflow.genai.scorers import (
    RelevanceToQuery,
    Correctness,
    Guidelines,
    Safety
)

if use_databricks_provider:
    databricks_token = os.environ.get("DATABRICKS_TOKEN")
    ai_gateway_base_url = os.environ.get("AI_GATEWAY_BASE_URL")
    
    # Try configuring as OpenAI-compatible endpoint
    os.environ["OPENAI_API_KEY"] = databricks_token
    os.environ["OPENAI_API_BASE"] = ai_gateway_base_url
    
    judge_model_uri = f"databricks:/{JUDGE_MODEL}"
    print("🔧 Configured for Databricks AI Gateway")
    print(f"   Base URL: {ai_gateway_base_url}")
    print(f"   Model: {JUDGE_MODEL}")
else:
    judge_model_uri = f"openai:/{JUDGE_MODEL}"

print(f"🔧 Judge model URI: {judge_model_uri}")

# Initialize built-in scorers
relevance_scorer = RelevanceToQuery(model=judge_model_uri)
correctness_scorer = Correctness(model=judge_model_uri)
safety_scorer = Safety(model=judge_model_uri)
guidelines_scorer = Guidelines(
    model=judge_model_uri,
    guidelines=[ """
Response should be appropriately detailed:
- Simple factual questions: < 200 words  
- Technical how-to questions: < 500 words
- Complex architectural questions: < 1000 words
"""
    ],
    name="custom_guidelines"
)

print("\n✅ Built-in scorers initialized:")
print("   - RelevanceToQuery, Correctness, Safety, Guidelines")

🔧 Judge model URI: openai:/gpt-5.2

✅ Built-in scorers initialized:
   - RelevanceToQuery, Correctness, Safety, Guidelines


---
## Step 5: Create Evaluation Dataset

This comprises three steps:
 1. Create an evaluation set that we will use in our evalution of the metrics to judge against the answer
 2. Create a prediction function that will take question as in input, and return a LLM response to the question
 3. run mlflow evaluation with all the scorers

### Why Create an Evaluation Dataset?

An evaluation dataset is the foundation of systematic agent testing. Without it, you're essentially "eyeballing" outputs—which doesn't scale and misses edge cases. The dataset enables **repeatable, automated evaluation** so you can detect regressions when you change prompts, models, or agent logic.

### What Purpose Does It Serve?

| Purpose | Benefit |
|---------|---------|
| **Benchmark Performance** | Measure how well your agent performs on known questions |
| **Detect Regressions** | Catch quality drops when updating prompts or models |
| **Compare Configurations** | A/B test different models, temperatures, or prompts |
| **Validate Edge Cases** | Include tricky questions that previously caused failures |
| **Document Expected Behavior** | Serve as living documentation of what your agent should do |

### How Many Evaluation Pairs?

**Rule of thumb: Start with 20-50 examples for development, scale to 100-500 for production.**

- **Minimum viable**: 10-20 pairs to catch obvious issues
- **Development**: 20-50 pairs covering main use cases
- **Pre-production**: 50-100 pairs including edge cases
- **Production monitoring**: 100-500+ pairs for statistical significance

The dataset below uses 8 examples for tutorial brevity, but real evaluations need more coverage.

### Dataset Structure

- `inputs`: The questions/prompts sent to the agent (dict with your input field names)
- `expectations` (optional): Expected answers for correctness checking

**Important:** Use `expected_response` as the field name in expectations—this is what MLflow's `Correctness` scorer looks for. Alternative: use `expected_facts` for fact-based evaluation.

In [6]:
# Evaluation dataset focused on MLflow for GenAI and Agent Observability
# NOTE: Use 'expected_response' - this is the field name that MLflow's Correctness scorer expects

eval_dataset = [
    {
        "inputs": {"question": "What is MLflow Tracing and why is it important for GenAI applications?"},
        "expectations": {"expected_response": "MLflow Tracing provides observability for GenAI applications by capturing the complete execution flow including LLM calls, retrieval steps, tool usage, and agent reasoning. It's important because it enables debugging, performance analysis, and understanding of complex AI pipelines."}
    },
    {
        "inputs": {"question": "How does MLflow help with prompt management in GenAI development?"},
        "expectations": {"expected_response": "MLflow's Prompt Registry allows you to version control prompts, tag and search prompt versions, link prompts to experiments, and collaborate with teams. This ensures reproducibility and systematic prompt engineering."}
    },
    {
        "inputs": {"question": "What are spans in MLflow Tracing and what types are available?"},
        "expectations": {"expected_response": "Spans are units of work captured during tracing. MLflow supports span types including LLM (for model calls), RETRIEVER (for RAG retrieval), TOOL (for function calls), AGENT (for agent orchestration), CHAIN (for sequential operations), and EMBEDDING (for vector operations)."}
    },
    {
        "inputs": {"question": "How can you evaluate LLM outputs using MLflow?"},
        "expectations": {"expected_response": "MLflow provides an evaluation framework with built-in scorers like RelevanceToQuery, Correctness, Guidelines, and Safety. You can also create custom scorers using the @scorer decorator or integrate third-party libraries like DeepEval and RAGAS."}
    },
    {
        "inputs": {"question": "What is the LLM-as-Judge pattern and how does MLflow support it?"},
        "expectations": {"expected_response": "LLM-as-Judge uses an LLM to evaluate outputs from another LLM, replacing brittle string matching with intelligent assessment. MLflow supports this through built-in scorers that use configurable judge models to provide scores and reasoning explanations."}
    },
    {
        "inputs": {"question": "How do you track costs and token usage in MLflow for GenAI?"},
        "expectations": {"expected_response": "MLflow automatically logs token usage (prompt tokens, completion tokens, total tokens) and can calculate costs based on model pricing. This data is captured in traces and experiment runs, enabling cost analysis and optimization across different models and configurations."}
    },
    {
        "inputs": {"question": "What frameworks does MLflow integrate with for GenAI auto-tracing?"},
        "expectations": {"expected_response": "MLflow provides auto-tracing for 40+ frameworks including OpenAI, Anthropic, LangChain, LlamaIndex, AWS Bedrock, Google Vertex AI, Cohere, Ollama, DSPy, AutoGen, and CrewAI. Auto-tracing automatically captures LLM calls without manual instrumentation."}
    },
    {
        "inputs": {"question": "How do you implement session-level tracing for multi-turn conversations?"},
        "expectations": {"expected_response": "Key concepts: (1) stable session identifier, (2) tag traces with session_id, (3) filter traces by session, (4) MLflow search capabilities"}
    },
]

print(f"✅ Evaluation dataset created with {len(eval_dataset)} examples")
print("\n📋 Questions cover:")
print("   - MLflow Tracing fundamentals")
print("   - Prompt management")
print("   - Span types and structure")
print("   - LLM evaluation methods")
print("   - LLM-as-Judge pattern")
print("   - Cost and token tracking")
print("   - Framework integrations")
print("   - Session-level observability")

✅ Evaluation dataset created with 8 examples

📋 Questions cover:
   - MLflow Tracing fundamentals
   - Prompt management
   - Span types and structure
   - LLM evaluation methods
   - LLM-as-Judge pattern
   - Cost and token tracking
   - Framework integrations
   - Session-level observability


### How predict_fn works?

When `mlflow.genai.evaluate()` runs:

1. It iterates through each item in `eval_dataset`
2. Extracts item["inputs"] and passes it to `predict_fn(inputs)`
3. Your function returns outputs (e.g., {"response": "..."})

Scorers then evaluate using inputs, outputs, and expectations
 `[inputs, outputs responses (from llm), expecttions] --> scorers`

 - **eval_dataset**: Defines what to test (questions + expected answers)
 - **predict_fn**: Defines how to get answers (calls your agent)
 - **scorers**: Define how to judge quality

In [7]:
def predict_fn(question: str) -> dict:
    """
    Prediction function wrapper for evaluation.
    
    Note: mlflow.genai.evaluate() unpacks the 'inputs' dict as keyword arguments,
    so the function signature must match the keys in your dataset's 'inputs' field.
    
    Dataset: {"inputs": {"question": "..."}} 
    Called as: predict_fn(question="...")
    
    Args:
        question: The question string (unpacked from inputs dict)
        
    Returns:
        Dictionary with 'response' key
    """
    response = agent.answer(question)
    return {"response": response}

print("✅ Prediction function defined")
print("   Signature: predict_fn(question: str) -> dict")

✅ Prediction function defined
   Signature: predict_fn(question: str) -> dict


---
## Step 6: Run Evaluation with Built-in Scorers

In [10]:
# Run evaluation
print("🔄 Running evaluation with built-in scorers...\n")

results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_fn,
    scorers=[
        relevance_scorer,
        correctness_scorer,
        safety_scorer,
        guidelines_scorer
    ]
)

print("\n✅ Evaluation complete!")

# Display metrics
print("\n📊 Metrics Summary:")
print("-" * 50)
if results.metrics:
    for metric_name, value in results.metrics.items():
        if isinstance(value, float):
            print(f"  {metric_name}: {value:.3f}")
        else:
            print(f"  {metric_name}: {value}")
else:
    print("  No metrics returned")
    print("\n⚠️  Scorers returned None - this usually means the judge model call failed.")
    print("  Check the 'error_message' or similar columns above for details.")

2026/02/06 12:00:47 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.


🔄 Running evaluation with built-in scorers...



Evaluating:   0%|          | 0/8 [Elapsed: 00:00, Remaining: ?] 


✅ Evaluation complete!

📊 Metrics Summary:
--------------------------------------------------
  safety/mean: 1.000
  relevance_to_query/mean: 1.000
  custom_guidelines/mean: 0.625
  correctness/mean: 0.250


---
## Step 7: Custom Scorers with @scorer Decorator

Create your own evaluation logic using the `@scorer` decorator.

Beyond using built-in metrics, you can define custom scorers to capture specific subject matter expertise. This is particularly useful when standard scorers cannot effectively gauge unique or nuanced aspects of your model's responses. While this example uses simple scorers for brevity and demonstration, you should tailor your custom metrics to reflect the specialized requirements of your specific domain.

In [11]:
from mlflow.genai import scorer

@scorer
def response_length_check(outputs: dict) -> bool:
    """
    Check if response is within acceptable length.
    Returns True if response is between 200 and 500 characters.
    """
    response = outputs.get("response", "")
    length = len(response)
    return 20 <= length <= 500

@scorer
def contains_keywords(outputs: dict, expectations: dict) -> bool:
    """
    Check if response contains key terms from expected answer.
    """
    response = outputs.get("response", "").lower()
    # Use 'expected_response' to match the dataset field name
    expected = expectations.get("expected_response", "").lower()
    
    # Extract key words (simple approach)
    key_words = [word for word in expected.split() if len(word) > 4]
    
    # Check if at least 30% of key words are present
    # If no keywords to check, fail conservatively (may indicate data issue)
    if not key_words:
        return False
    
    matches = sum(1 for word in key_words if word in response)
    return matches / len(key_words) >= 0.3

@scorer
def no_hallucination_markers(outputs: dict) -> bool:
    """
    Check for common hallucination markers.
    """
    response = outputs.get("response", "").lower()
    
    hallucination_markers = [
        "i think",
        "i believe",
        "probably",
        "might be",
        "i'm not sure",
        "as far as i know"
    ]
    
    return not any(marker in response for marker in hallucination_markers)

print("✅ Custom scorers defined:")
print("   - response_length_check")
print("   - contains_keywords")
print("   - no_hallucination_markers")

✅ Custom scorers defined:
   - response_length_check
   - contains_keywords
   - no_hallucination_markers


### Run evaluation on our Customer Judges

In [12]:
# Run evaluation with custom scorers
print("🔄 Running evaluation with custom scorers...\n")

custom_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_fn,
    scorers=[
        response_length_check,
        contains_keywords,
        no_hallucination_markers
    ]
)

print("\n✅ Custom evaluation complete!")
print("\nCustom Metrics Summary:")
print("-" * 40)
for metric_name, value in custom_results.metrics.items():
    if isinstance(value, float):
        print(f"  {metric_name}: {value:.3f}")
    else:
        print(f"  {metric_name}: {value}")

2026/02/06 12:06:01 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.


🔄 Running evaluation with custom scorers...



Evaluating:   0%|          | 0/8 [Elapsed: 00:00, Remaining: ?] 


✅ Custom evaluation complete!

Custom Metrics Summary:
----------------------------------------
  response_length_check/mean: 0.125
  no_hallucination_markers/mean: 1.000
  contains_keywords/mean: 0.750


---
## Step 8: DeepEval Integration

MLflow integrates with [DeepEval](https://mlflow.org/docs/latest/genai/eval-monitor/scorers/third-party/deepeval/) for advanced conversational AI evaluation metrics.

| DeepEval Scorer | Purpose |
|-----------------|----------|
| `ConversationCompleteness` | Did the conversation achieve its goal? |
| `KnowledgeRetention` | Does the agent remember context? |
| `TopicAdherence` | Does the agent stay on topic? |
| `Toxicity` | Is the agent response harmful or toxic in tonality|

In [14]:
from mlflow.genai.scorers.deepeval import (
    ConversationCompleteness,
    KnowledgeRetention,
    TopicAdherence,
    Toxicity,
)

# Initialize DeepEval scorers
jude_model_uri = f"openai:/{JUDGE_MODEL}"

completeness_scorer = ConversationCompleteness(model=jude_model_uri, threshold=0.7, include_reason=True)
retention_scorer = KnowledgeRetention(model=jude_model_uri, threshold=0.7, include_reason=True)
toxicity_scorer = Toxicity(model=jude_model_uri, threshold=0.7, include_reason=True)
topic_scorer = TopicAdherence(model=jude_model_uri, threshold=0.7, include_reason=True, relevant_topics=["MLflow", "machine learning", "AI", "data science", "genai", "agent", "observability", "prompt engineering", "prompt management", "prompt registry", "experiment tracking"])


print("✅ DeepEval scorers initialized:")
print("   - ConversationCompleteness")
print("   - KnowledgeRetention")
print("   - TopicAdherence")
print("   - Toxicity")
print("\n Next, let's evaluate a multi-turn conversation using these DeepEval scorers")

✅ DeepEval scorers initialized:
   - ConversationCompleteness
   - KnowledgeRetention
   - TopicAdherence
   - Toxicity

 Next, let's evaluate a multi-turn conversation using these DeepEval scorers


---
## Step 9: Multi-Turn Conversation Agent

For session-level evaluation, we need an agent that handles conversations.

In [16]:
import uuid

class ConversationalAgent:
    """
    An agent that maintains conversation history for multi-turn interactions.
    """
    
    def __init__(self, client: Any, model: str = "gpt-5.2"):
        self.model = model
        self.client = client
        self.conversation_history = []
        self.session_id = str(uuid.uuid4())
    
    def reset(self):
        """Reset conversation history and start new session."""
        self.conversation_history = []
        self.session_id = str(uuid.uuid4())
    
    @mlflow.trace(name="conversational_agent", span_type="AGENT")
    def chat(self, user_message: str) -> str:
        """
        Send a message and get a response, maintaining history.
        """
        # Tag trace with session ID for grouping
        mlflow.update_current_trace(metadata={
            "mlflow.trace.session": self.session_id,
            "turn_number": len(self.conversation_history) // 2 + 1
        })
        
        # Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        # Prepare messages with system prompt. Add previous context to the conversation.
        messages = [
            {"role": "system", 
            "content": """You are a helpful MLflow expert assistant. Answer questions about MLflow clearly, accurately,
                        concisely, without hallucinations, and accurately. Remember previous context in the conversation."""}
        ] + self.conversation_history
        
        # Get response from the Agent LLM
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.7,
        )
        
        assistant_message = response.choices[0].message.content
        
        # Add to history
        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return assistant_message

print("✅ ConversationalAgent defined with session tracking")

✅ ConversationalAgent defined with session tracking


### Simulate a multi-turn conversation

In [17]:
# Simulate a multi-turn conversation
conv_agent = ConversationalAgent(client=client, model=AGENT_MODEL)

conversation_turns = [
    "What is MLflow for GenAI?",
    "What are its main GenAI main components?",
    "Tell me more about the Tracing component.",
    "How does it compare to other tools?",
    "What is the difference between Tracing and Tracking?",
    "How do I get started with MLflow for GenAI?",
]

print("🗣️ Multi-Turn Conversation\n")
print("=" * 60)

for i, user_msg in enumerate(conversation_turns, 1):
    print(f"\n[Turn {i}]")
    print(f"User: {user_msg}")
    response = conv_agent.chat(user_msg)
    print(f"Agent: {response}")

print("\n" + "=" * 60)
print(f"\n✅ Conversation complete (Session: {conv_agent.session_id[:8]}...)")

2026/02/06 12:12:52 WARNING mlflow.tracing.fluent: Found non-string values in metadata. Non-string values in metadata will automatically be stringified when the trace is logged. Non-string items: {'turn_number': 1}


🗣️ Multi-Turn Conversation


[Turn 1]
User: What is MLflow for GenAI?


2026/02/06 12:12:59 WARNING mlflow.tracing.fluent: Found non-string values in metadata. Non-string values in metadata will automatically be stringified when the trace is logged. Non-string items: {'turn_number': 2}


Agent: MLflow for GenAI is the set of MLflow capabilities focused on building, evaluating, tracking, and deploying **generative AI applications** (LLM-powered apps), not just traditional ML models.

In practice, it helps you manage things like **prompts, models, retrieval components, and outputs** with the same lifecycle tooling MLflow provides for ML.

Key pieces include:

- **Prompt & experiment tracking**  
  Track runs that include prompts, system messages, model/provider (e.g., OpenAI, Anthropic, local models), parameters (temperature, max tokens), tool calls, and outputs—so you can reproduce and compare changes.

- **LLM evaluation (quality, safety, and regression testing)**  
  Evaluate LLM apps using built-in evaluators and/or custom metrics (e.g., correctness, relevance, faithfulness, toxicity, latency, cost). This supports automated comparisons across prompt versions or model swaps.

- **Model packaging & deployment for LLM apps**  
  Package GenAI logic (prompt + code + depe

2026/02/06 12:13:04 WARNING mlflow.tracing.fluent: Found non-string values in metadata. Non-string values in metadata will automatically be stringified when the trace is logged. Non-string items: {'turn_number': 3}


...

2026/02/06 12:13:29 WARNING mlflow.tracing.fluent: Found non-string values in metadata. Non-string values in metadata will automatically be stringified when the trace is logged. Non-string items: {'turn_number': 5}


Agent: MLflow Tracing is a **general-purpose, ML lifecycle–integrated** tracing option for GenAI apps. Compared to dedicated LLM observability products (LangSmith, Arize Phoenix, Weights & Biases Weave, Helicone, Humanloop, etc.) it tends to be strongest when you want **one system** for tracking/eval/registry/deployment plus tracing, and you’re already using MLflow/Databricks.

Here are the main comparison axes:

## 1) Scope: “LLM observability” vs “ML lifecycle”
- **MLflow Tracing**: Tracing is one component alongside **Tracking, Evaluation, Model Registry, and Serving**. Good if you want traces tied directly to runs, evaluations, and model/app versions.
- **Dedicated LLM observability tools**: Often go deeper on observability workflows (prompt playgrounds, annotation queues, dataset curation loops, specialized dashboards), but may be a separate system from your ML model registry/deployment.

## 2) Framework integrations & auto-instrumentation
- **MLflow Tracing**: Works best when you

2026/02/06 12:13:37 WARNING mlflow.tracing.fluent: Found non-string values in metadata. Non-string values in metadata will automatically be stringified when the trace is logged. Non-string items: {'turn_number': 6}


Agent: **Tracking** and **Tracing** solve different problems in MLflow:

## MLflow Tracking (experiment/run tracking)
**Purpose:** Record and compare *experiments* and their outcomes.

**Typical unit:** an **MLflow Run** (one experiment attempt).

**What you log:**
- **Parameters** (e.g., temperature, top_p, chunk size, prompt version)
- **Metrics** (e.g., accuracy, win rate, faithfulness score, latency aggregates)
- **Artifacts** (prompt templates, evaluation datasets, model/app code, configs)
- **Tags/metadata** (git SHA, dataset version, environment)

**What it’s good for:**
- Reproducibility and auditability of an experiment
- Comparing variants (prompt A vs B, model X vs Y)
- Capturing evaluation results and dashboards over many runs

## MLflow Tracing (execution-level observability)
**Purpose:** Understand *what happened inside* a single LLM app execution.

**Typical unit:** a **Trace** made of **spans** (steps).

**What it captures (per step):**
- Inputs/outputs for each step (p

[Trace(trace_id=tr-e3feeb5a5698e9aab95f9f2a076ed659), Trace(trace_id=tr-82aa410f6ae2970481a8c9b5a976bf05), Trace(trace_id=tr-2b45100523f045ee2f712519608daa3b), Trace(trace_id=tr-c7f5188e41d8086d806f9c075f81c07b), Trace(trace_id=tr-fcef8024c8e02fefa232a8a8571ce35e), Trace(trace_id=tr-d34e3612996ec65c1422f872dfbedd78)]

---
## Step 10: Session-Level Evaluation with DeepEval

In [19]:
# Search for traces from this session
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

session_traces = mlflow.search_traces(
    locations=[experiment.experiment_id],
    filter_string=f"metadata.`mlflow.trace.session` = '{conv_agent.session_id}'"
)

print(f"📊 Found {len(session_traces)} traces for session {conv_agent.session_id[:8]}...")

📊 Found 6 traces for session 2f3a78f7...


### Run the evaluation

In [21]:
with mlflow.start_run(run_name="DeepEval") as run:
    deepeval_results = mlflow.genai.evaluate(
        data= session_traces,
        scorers=[
            completeness_scorer,
            retention_scorer,
            topic_scorer,
            toxicity_scorer
        ]
    )

print("\n✅ DeepEval session evaluation complete!")
print("\nDeepEval Metrics:")
print("-" * 40)
for metric_name, value in deepeval_results.metrics.items():
    if isinstance(value, float):
        print(f"  {metric_name}: {value:.3f}")
    else:
        print(f"  {metric_name}: {value}")

Evaluating:   0%|          | 0/7 [Elapsed: 00:00, Remaining: ?] 


✅ DeepEval session evaluation complete!

DeepEval Metrics:
----------------------------------------
  Toxicity/mean: 1.000
  KnowledgeRetention/mean: 0.000
  ConversationCompleteness/mean: 1.000
  TopicAdherence/mean: 1.000


---
## Step 11: Viewing Evaluation Results in MLflow UI

In [33]:
print("""
╔══════════════════════════════════════════════════════════════╗
║         Viewing Evaluation Results in MLflow UI              ║
╚══════════════════════════════════════════════════════════════╝

🔍 EXPERIMENTS VIEW:
   Navigate to: http://localhost:5000
   Select: "11-agent-evaluation" experiment
   
   You'll see:
   - Session-level and Evaluation runs with scorer metrics
   - Pass/fail rates for each scorer
   - Detailed reasoning from LLM judges

📊 EVALUATION RESULTS:
   Each run includes:
   - Metric values (0-1 scores or boolean)
   - Judge reasoning explanations
   - Input/output pairs
   - Artifacts with detailed results

🎯 KEY METRICS TO MONITOR:
   
   Built-in Scorers:
   - relevance_to_query/score: Response relevance
   - correctness/score: Factual accuracy
   - safety/score: Safety compliance
   - guidelines/score: Guideline adherence
   
   Custom Scorers:
   - response_length_check: Length validation
   - contains_keywords: Keyword presence
   - no_hallucination_markers: Confidence check
   
   DeepEval Scorers:
   - conversation_completeness/score: Goal achievement
   - knowledge_retention/score: Context memory
   - topic_adherence/score: Topic focus

💡 TIPS:
   1. Compare multiple evaluation runs
   2. Filter by scorer to find failures
   3. Read judge reasoning for insights
   4. Track metrics over time for regression detection

""")


╔══════════════════════════════════════════════════════════════╗
║         Viewing Evaluation Results in MLflow UI              ║
╚══════════════════════════════════════════════════════════════╝

🔍 EXPERIMENTS VIEW:
   Navigate to: http://localhost:5000
   Select: "11-agent-evaluation" experiment

   You'll see:
   - Session-level and Evaluation runs with scorer metrics
   - Pass/fail rates for each scorer
   - Detailed reasoning from LLM judges

📊 EVALUATION RESULTS:
   Each run includes:
   - Metric values (0-1 scores or boolean)
   - Judge reasoning explanations
   - Input/output pairs
   - Artifacts with detailed results

🎯 KEY METRICS TO MONITOR:

   Built-in Scorers:
   - relevance_to_query/score: Response relevance
   - correctness/score: Factual accuracy
   - safety/score: Safety compliance
   - guidelines/score: Guideline adherence

   Custom Scorers:
   - response_length_check: Length validation
   - contains_keywords: Keyword presence
   - no_hallucination_markers: Confiden

---
## Summary

In this notebook, you learned how to evaluate agents and LLM applications using MLflow's evaluation framework.

### ✅ What You Learned

**Built-in Scorers:**
- `RelevanceToQuery` - Check if responses are relevant
- `Correctness` - Verify factual accuracy
- `Guidelines` - Enforce custom guidelines
- `Safety` - Ensure safe outputs

**Custom Scorers:**
- Use `@scorer` decorator for custom logic
- Access `outputs` and `expectations` in scorers
- Return boolean or numeric scores

**DeepEval Integration:**
- `ConversationCompleteness` - Goal achievement
- `KnowledgeRetention` - Context memory
- `TopicAdherence` - Topic focus

**Session-Level Evaluation:**
- Tag traces with session IDs
- Search traces by session
- Evaluate multi-turn conversations

### 🔑 Key Patterns

```python
# Built-in scorer
from mlflow.genai.scorers import RelevanceToQuery
scorer = RelevanceToQuery(model="openai:/gpt-4o-mini")

# Custom scorer
from mlflow.genai import scorer
@scorer
def my_scorer(outputs) -> bool:
    return condition

# Run evaluation
results = mlflow.genai.evaluate(
    data=session_tracew,
    scorers=[scorer1, scorer2]
)
```

### 📚 Next Steps

Continue to **Tutorial 1.8: Complete RAG Application** to build a full RAG system with RAGAS evaluation metrics.